#### Business Problem and Dataset

* This case and dataset is taken from the Recommender Systems course on www.miuul.com. 

* The dataset named Online Retail II includes online sales transactions of a UK-based retail company between 01/12/2009 and 09/12/2011. The company's product catalog includes souvenirs and it is known that most of its customers are wholesalers.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
from mlxtend.frequent_patterns import apriori, association_rules
sns.set(style="ticks")
sns.set_style("darkgrid")
!pip install openpyxl
dff= pd.read_excel("../input/onineretaildata/online_retail_II.xlsx", 
                   sheet_name="Year 2010-2011",engine="openpyxl")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 1.6 MB/s eta 0:00:00


In [2]:
df=dff.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
def check_df(df):
    print("------- Shape ----------")
    print(df.shape)
    print("------- Dtypes ----------")
    print(df.dtypes)
    print("------- NA ----------")
    print(df.isnull().sum().sort_values(ascending=False))
check_df(df)

------- Shape ----------
(541910, 8)
------- Dtypes ----------
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
------- NA ----------
Customer ID    135080
Description      1454
Invoice             0
StockCode           0
Quantity            0
InvoiceDate         0
Price               0
Country             0
dtype: int64


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


* As seen in the summary statistics, the minimum values are negative. The reason for this is the canceled orders. The canceled orders are the orders with the initial letter "C" in the invoice variable in the dataset.We need to continue by removing these canceled orders from the dataset.To ensure getting rid of negative values, we should only select price and quantity variables greater than 0. We should also drop missing values.

In [5]:
def retail_data_prep(df):
    df.dropna(inplace=True)
    df=df[~(df["StockCode"]=="POST")]
    df=df[~df["Invoice"].str.contains("C",na=False)]
    df = df[df["Quantity"] > 0]
    df = df[df["Price"] > 0]
    return df
df=retail_data_prep(df)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,13.016349,179.579125,1.000,2.00,6.00,12.00,80995.00
Price,396785.0,3.037677,17.829741,0.001,1.25,1.95,3.75,4161.06
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.00


* When we examine the new dataset created after these operations, we can see that the negative values are deleted. But when we look at the quarterly and maximum values of the and price and quantitiy variables, it is seen that they have outliers. We can use the suppression method to solve this problem.In the suppression method, outliers are replaced with upper limit and lower limits.

In [7]:
# determine the upper and lower limit
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [8]:
# replacing outlirs with upper and lower limit
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [9]:
for i in ["Quantity","Price"]:
    replace_with_thresholds(df,i)

In [10]:
df.describe().T
# As seen with the describe function, the outliers are greatly reduced.

,count,mean,std,min,25%,50%,75%,max
Quantity,396785.0,11.855703,25.553760,1.000,2.00,6.00,12.00,298.50
Price,396785.0,2.835605,2.982371,0.001,1.25,1.95,3.75,31.56
Customer ID,396785.0,15301.463886,1709.852311,12346.000,13975.00,15159.00,16801.00,18287.00


 * In the Association Rule Learning data structure, invoice or cart should be placed on the rows and products should be placed on the columns. At the intersection of these, there should be information about whether that product exists in that invoice or not.This is expressed by 0 and 1. In this project, we will create a recommendation system specific to Germany.

In [11]:
df_gr = df[df['Country'] == "Germany"]

In [12]:
df_gr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().head()

Quantity                     \
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER   
Invoice                                                         
536527                                 NaN                NaN   
536840                                 NaN                NaN   
536861                                 NaN                NaN   
536967                                 NaN                NaN   
536983                                 NaN                NaN   

                                                                   \
Description  I LOVE LONDON MINI BACKPACK  RED SPOT GIFT BAG LARGE   
Invoice                                                             
536527                               NaN                      NaN   
536840                               NaN                      NaN   
536861                               NaN                      NaN   
536967                               NaN                      NaN   
536983                               NaN                      NaN   

                                                                      \
Description  SET 2 TEA TOWELS I LOVE LONDON   SPACEBOY BABY GIFT SET   
Invoice                                                                
536527                                   NaN                     NaN   
536840                                   NaN                     NaN   
536861                                   NaN                     NaN   
536967                                   NaN                     NaN   
536983                                   NaN                     NaN   

                                                               \
Description 10 COLOUR SPACEBOY PEN 12 COLOURED PARTY BALLOONS   
Invoice                                                         
536527                         NaN                        NaN   
536840                         NaN                        NaN   
536861                         NaN                        NaN   
536967                         NaN                        NaN   
536983                         NaN                        NaN   

                                                                              \
Description 12 IVORY ROSE PEG PLACE SETTINGS 12 MESSAGE CARDS WITH ENVELOPES   
Invoice                                                                        
536527                                   NaN                             NaN   
536840                                   NaN                             NaN   
536861                                   NaN                             NaN   
536967                                   NaN                             NaN   
536983                                   NaN                             NaN   

             ...                                                           \
Description  ... YULETIDE IMAGES GIFT WRAP SET ZINC  HEART T-LIGHT HOLDER   
Invoice      ...                                                            
536527       ...                           NaN                        NaN   
536840       ...                           NaN                        NaN   
536861       ...                           NaN                        NaN   
536967       ...                           NaN                        NaN   
536983       ...                           NaN                        NaN   

                                                           \
Description ZINC  STAR T-LIGHT HOLDER  ZINC BOX SIGN HOME   
Invoice                                                     
536527                             NaN                NaN   
536840                             NaN                NaN   
536861                             NaN                NaN   
536967                             NaN                NaN   
536983                             NaN                NaN   

                                                                         \
Description ZINC FOLKART SLEIGH BELLS ZI

* As mentioned, we created the dataset with product in the invoice columns in the rows. But there is an NA value in the products that do not exist. We need to replace these NAs with 0.
* Also, quantity values are written at the intersections, but what we want is whether that product exists in the invoice. Therefore, we need to convert cells containing a numeric value greater than zero to 1 using the applymap function.

In [13]:
df_gr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().fillna(0).head()

Quantity                     \
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER   
Invoice                                                         
536527                                 0.0                0.0   
536840                                 0.0                0.0   
536861                                 0.0                0.0   
536967                                 0.0                0.0   
536983                                 0.0                0.0   

                                                                   \
Description  I LOVE LONDON MINI BACKPACK  RED SPOT GIFT BAG LARGE   
Invoice                                                             
536527                               0.0                      0.0   
536840                               0.0                      0.0   
536861                               0.0                      0.0   
536967                               0.0                      0.0   
536983                               0.0                      0.0   

                                                                      \
Description  SET 2 TEA TOWELS I LOVE LONDON   SPACEBOY BABY GIFT SET   
Invoice                                                                
536527                                   0.0                     0.0   
536840                                   0.0                     0.0   
536861                                   0.0                     0.0   
536967                                   0.0                     0.0   
536983                                   0.0                     0.0   

                                                               \
Description 10 COLOUR SPACEBOY PEN 12 COLOURED PARTY BALLOONS   
Invoice                                                         
536527                         0.0                        0.0   
536840                         0.0                        0.0   
536861                         0.0                        0.0   
536967                         0.0                        0.0   
536983                         0.0                        0.0   

                                                                              \
Description 12 IVORY ROSE PEG PLACE SETTINGS 12 MESSAGE CARDS WITH ENVELOPES   
Invoice                                                                        
536527                                   0.0                             0.0   
536840                                   0.0                             0.0   
536861                                   0.0                             0.0   
536967                                   0.0                             0.0   
536983                                   0.0                             0.0   

             ...                                                           \
Description  ... YULETIDE IMAGES GIFT WRAP SET ZINC  HEART T-LIGHT HOLDER   
Invoice      ...                                                            
536527       ...                           0.0                        0.0   
536840       ...                           0.0                        0.0   
536861       ...                           0.0                        0.0   
536967       ...                           0.0                        0.0   
536983       ...                           0.0                        0.0   

                                                           \
Description ZINC  STAR T-LIGHT HOLDER  ZINC BOX SIGN HOME   
Invoice                                                     
536527                             0.0                0.0   
536840                             0.0                0.0   
536861                             0.0                0.0   
536967                             0.0                0.0   
536983                             0.0                0.0   

                                                                         \
Description ZINC FOLKART SLEIGH BELLS ZI

In [14]:
gr_inv_pro_df = df_gr.groupby(['Invoice', 'StockCode']). \
    agg({"Quantity": "sum"}). \
    unstack(). \
    fillna(0). \
    applymap(lambda x: 1 if x > 0 else 0).head()

* Made with stock code instead of description for easier readability.As a result of these all procedures the data set has come to a structure that can be used for association rule.

In [15]:
frequent_itemsets = apriori(gr_inv_pro_df,
                            min_support=0.02,
                            use_colnames=True)

In [16]:
frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
8,0.4,"((Quantity, 22077))"
0,0.2,"((Quantity, 16016))"
12112,0.2,"((Quantity, 22549), (Quantity, 22243), (Quanti..."
12118,0.2,"((Quantity, 22549), (Quantity, 22837), (Quanti..."
12117,0.2,"((Quantity, 22549), (Quantity, 22837), (Quanti..."
...,...,...
6064,0.2,"((Quantity, 22837), (Quantity, 22549), (Quanti..."
6065,0.2,"((Quantity, 22549), (Quantity, 22243), (Quanti..."
6066,0.2,"((Quantity, 22549), (Quantity, 22243), (Quanti..."
6067,0.2,"((Quantity, 84347), (Quantity, 22549), (Quanti..."


* The apriori function evaluated based on the support value, but we want to evaluate and filter based on the confidence and lift values as well. That's why we will use the association_rules function.

In [17]:
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

In [18]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"((Quantity, 22171))","((Quantity, 16016))",0.2,0.2,0.2,1.0,5.0,0.16,inf
1,"((Quantity, 16016))","((Quantity, 22171))",0.2,0.2,0.2,1.0,5.0,0.16,inf
2,"((Quantity, 22331))","((Quantity, 16016))",0.2,0.2,0.2,1.0,5.0,0.16,inf
3,"((Quantity, 16016))","((Quantity, 22331))",0.2,0.2,0.2,1.0,5.0,0.16,inf
4,"((Quantity, 16016))","((Quantity, 22445))",0.2,0.2,0.2,1.0,5.0,0.16,inf
...,...,...,...,...,...,...,...,...,...
4831909,"((Quantity, 47421))","((Quantity, 84347), (Quantity, 22549), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf
4831910,"((Quantity, 84945))","((Quantity, 84347), (Quantity, 22549), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf
4831911,"((Quantity, 84569B))","((Quantity, 84347), (Quantity, 22549), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf
4831912,"((Quantity, 22244))","((Quantity, 84347), (Quantity, 22549), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf


* all possible combinations, namely association rules, are listed above. Approximately 5 million association rules have been created. If we want, we can determine and filter a treshold according to the specified suuport, confidence and lift values.

In [19]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>3)].sort_values("confidence").head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"((Quantity, 22171))","((Quantity, 16016))",0.2,0.2,0.2,1.0,5.0,0.16,inf
3221787,"((Quantity, 84347), (Quantity, 22837), (Quanti...","((Quantity, 22244))",0.2,0.2,0.2,1.0,5.0,0.16,inf
3221786,"((Quantity, 84347), (Quantity, 22837), (Quanti...","((Quantity, 22809))",0.2,0.2,0.2,1.0,5.0,0.16,inf
3221785,"((Quantity, 22969))","((Quantity, 84347), (Quantity, 22837), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf
3221784,"((Quantity, 22244))","((Quantity, 84347), (Quantity, 22837), (Quanti...",0.2,0.2,0.2,1.0,5.0,0.16,inf


* Let's briefly interpret the above table and metrics
* antecedents : represents the first product or product set
* consequents : represents the second product or product set
* antecedent support : frequency of the first product or product set
* consequent support : frequency of the second product or product set
* support : frequency of coexistence of two products or product sets
* confidence : The probability of receiving the second product or product set when the first product or product set is purchased
* lift : Represents the rate of increase in purchasing the second product or product set when the first product or product set is purchased.

* With the defined check_id function, we can easily find the detailed name of a product whose stock_code we enter.

In [20]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    
# example
check_id(df_gr, 22492)

['MINI PAINT SET VINTAGE ']


With the function below, it is possible to find which product is recommended according to the product selected by the user.

In [21]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]